In [ ]:
# %pip install requests beautifulsoup4 pymupdf sentence-transformers faiss-cpu

In [ ]:
import requests, os, fitz, faiss
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

### 삼성증권 리포트 다운로드

In [ ]:
# 1. 요청
url = "https://www.samsungpop.com/sscommon/jsp/search/research/research_pop.jsp"
resp = requests.post(url)
soup = BeautifulSoup(resp.text, "html.parser")

# 2. fileName 추출
file_names = []
table = soup.find("table", class_="tbl-type board")
for a_tag in table.find_all("a", href=True, title=True):
    href = a_tag['href']
    if "fileName=" in href:
        file_name = href.split("fileName=")[-1]
        file_names.append(file_name)

# 3. PDF 다운로드
os.makedirs("samsungpop", exist_ok=True)
base_url = "http://www.samsungpop.com/common.do?cmd=down&saveKey=research.pdf&"

for file_name in file_names:
    pdf_url = base_url + "fileName=" + file_name
    filename = file_name.split("/")[-1]  # 파일명만 추출
    filepath = os.path.join("samsungpop", filename)
    r = requests.get(pdf_url)
    with open(filepath, "wb") as f:
        f.write(r.content)
    print(f"Downloaded: {filepath}")

Downloaded: samsungpop\2025091110204678K_02_02.pdf
Downloaded: samsungpop\2025091107171579K_02_03.pdf
Downloaded: samsungpop\2025091023090734K_02_02.pdf
Downloaded: samsungpop\2025090916113997K_02_04.pdf
Downloaded: samsungpop\2025091019084479K_02_03.pdf
Downloaded: samsungpop\2025091016461946K_02_02.pdf
Downloaded: samsungpop\2025091015481709K_02_02.pdf


### pdf 파일 읽기

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

def chunk_text(text, chunk_size=500):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i+chunk_size])

# pdf_folder 해당 폴더 경로 내 파일 이름 리스트로 저장
pdf_folder = "samsungpop"
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

texts = [extract_text_from_pdf(pdf) for pdf in pdf_files]
chunks = []
for t in texts:
    chunks.extend(list(chunk_text(t)))

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
# 검색 예시
query = "헬스케어 수출 데이터"
query_vector = model.encode([query])
D, I = index.search(query_vector, k=5)
print([chunks[i] for i in I[0]])

['리서치센터투자정보팀조아인선임연구원 매크로 테마 BF: 미국정규장전/ AF: 미국정규장후 기업 한줄코멘트 [실적] • BF)JP모건, 씨티그룹, 블랙록, 웰스파고 [실적] • ASML • BF)모건스탠리, 골드만삭스 BofA, 존슨앤존슨 AF)테슬라 [실적] • TSMC • BF)펩시코, US뱅코프 AF)넷플릭스 [실적] • 현대건설, 포스코퓨처엠 • BF)아메리칸익스프레스, 3M, 찰스슈왑 캘박 캘린더박제 3주 월 9 9월FOMC에서25bp 인하가기정사실화된가운데, 함께공개될점도표에시장은더민감하게반응할전망 [바이오] 유럽당뇨학회(EASD)(~9/19) [증시] 코리아인베스트먼트위크2025(~9/18) [디지털화폐] 기재부·한은, CBDC 사업설명회 [2차전지] 코리아어드밴스드배터리컨퍼런스 [국방] 중국샹산포럼(~9/19) [외교] 트럼프대통령, 영국방문(~9/19) [IT] 애플, 아이폰17 시리즈출시 삼성전자, 갤럭시S25 FE 국내출시 [2차전지] 오토살롱테크코리아(~9/21) • 일본증시휴장(경로의날) • 8월소매판매, 산업생산★(11:00) • 7월무역수지 • 9월뉴욕주제조업지수 • 8월수출입물가지수 • FOMC 회의(~9/17) ★ • 8월소매판매★(21:30) • 8월수출입물가지수 • 8월산업생산 • 20년만기국채입찰 • 8월무역수지 • 8월주택착공, 건축허가 • Fed 기준금리결정★ (18일03:00) • BOJ 금융정책결정회의(~9/19) • BOE 기준금리결정 • 주간실업수당청구건수 • 9월필라델피아연은제조업지수 • 8월컨퍼런스보드경기선행지수 • BOJ 기준금리결정★(12:00) • 선물·옵션동시만기일★ 15 월 16 화 17 수 18 목 19 금 ※ 현지시각기준(괄호안은우리시각) BF: 미국정규장전/ AF: 미국정규장후 기업 [실적] • AF) 페덱스 MON TUE WED THU FRI 9/22 9/23 9/24 9/25 9/26 한국9월1~20일수출입(9/21) 코리아블록체인위크(~9/28) 정부, 2차소비쿠폰지급 배터리코리아 일본증시휴